In [78]:
import re
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [79]:

# Function to parse the log file
def parse_log_file(log_file_path):
    iterations = []
    mious = []
    maccs = []
    learning_rates = []

    last_iter = None
    last_lr = None

    with open(log_file_path, 'r') as f:
        for line in f:
            # Parse training iteration and learning rate
            train_match = re.search(r"Iter\(train\)\s*\[\s*(\d+)/\d+\]\s*lr:\s*([0-9\.e+-]+)", line)
            if train_match:
                last_iter = int(train_match.group(1))
                last_lr = float(train_match.group(2))
                continue

            # Parse validation metrics (mIoU and mAcc)
            val_match = re.search(r"mIoU:\s*(\d+\.\d+)\s*mAcc:\s*(\d+\.\d+)", line)
            if val_match:
                if last_iter is not None:
                    iterations.append(last_iter)
                    mious.append(float(val_match.group(1)))
                    maccs.append(float(val_match.group(2)))
                    learning_rates.append(last_lr)

    return iterations, mious, maccs, learning_rates

# New function to plot interactively with Plotly
def plot_accuracies_and_lr_interactive(epochs, mAcc, mIoU, lrs, x_range):
    # Create a figure with secondary y-axis
    fig = make_subplots(specs=[[{"secondary_y": True}]])

    # Add Top-1 Accuracy trace
    fig.add_trace(
        go.Scatter(x=epochs, y=mAcc, name='Mean Accuracy', line=dict(color='blue')),
        secondary_y=False,
    )

    # Add Top-5 Accuracy trace
    fig.add_trace(
        go.Scatter(x=epochs, y=mIoU, name='Mean IoU', line=dict(color='orange')),
        secondary_y=False,
    )

    # Add Learning Rate trace on secondary y-axis
    fig.add_trace(
        go.Scatter(x=epochs, y=lrs, name='Learning Rate', line=dict(color='green')),
        secondary_y=True,
    )

    # Update layout for interactivity and axes
    fig.update_layout(
        title='Validation Performance and Training Learning Rate vs Epoch',
        xaxis_title='Epoch',
        yaxis_title='Performance (%)',
        yaxis2_title='Learning Rate',
        hovermode='x unified',  # Vertical crosshair on x-hover, shows values for all lines
        legend=dict(yanchor='top', y=1.15, xanchor='right', x=1.15),  # Top-right legend
    )

    # Set x-axis ranges
    fig.update_xaxes(range=x_range)
    
    # Set y-axis ranges
    fig.update_yaxes(range=[0, 100], dtick=10, secondary_y=False)
    fig.update_yaxes(secondary_y=True)  # Auto-range for LR

    # Show the figure (interactive in Jupyter)
    fig.show()


In [80]:
adam_range = [0, 310]
sgd_range = [0, 120000]

In [81]:
# Ablation 01 - SEBNet BaselineHead SGD, Pretrained, Batch Size 6, OHEM Cross Entropy - 72.72 Best mIoU
log_file_path = '/home/robert.breslin/alessandro/paper_2/mmsegmentation/work_dirs/sebnet_baseline-head_1xb6_cityscapes_ohem/20250822_162921/20250822_162921.log'
epochs, mIoU, mAcc, lrs = parse_log_file(log_file_path)
plot_accuracies_and_lr_interactive(epochs, mAcc, mIoU, lrs, [0,160000])

In [82]:
# Ablation 01 - SEBNet BaselineHead SGD, Scratch, Batch Size 6, OHEM Cross Entropy - 72.43 Best mIoU
log_file_path = '/home/robert.breslin/alessandro/paper_2/mmsegmentation/work_dirs/sebnet_baseline-head_1xb6_cityscapes_ohem_scratch/20250822_162814/20250822_162814.log'
epochs, mIoU, mAcc, lrs = parse_log_file(log_file_path)
plot_accuracies_and_lr_interactive(epochs, mAcc, mIoU, lrs, [0,160000])

In [83]:
# Ablation 01 - SEBNet BaselineHead SGD, Pretrained, Batch Size 8, OHEM Cross Entropy - 72.34 Best mIoU
log_file_path = '/home/robert.breslin/alessandro/paper_2/mmsegmentation/work_dirs/sebnet_baseline-head_1xb8_cityscapes_ohem/20250822_163056/20250822_163056.log'
epochs, mIoU, mAcc, lrs = parse_log_file(log_file_path)
plot_accuracies_and_lr_interactive(epochs, mAcc, mIoU, lrs, [0,120000])

In [84]:
# Ablation 01 - SEBNet BaselineHead SGD, Scratch, Batch Size 8, OHEM Cross Entropy - 72.7 Best mIoU
log_file_path = '/home/robert.breslin/alessandro/paper_2/mmsegmentation/work_dirs/sebnet_baseline-head_1xb8_cityscapes_ohem_scratch/20250822_163019/20250822_163019.log'
epochs, mIoU, mAcc, lrs = parse_log_file(log_file_path)
plot_accuracies_and_lr_interactive(epochs, mAcc, mIoU, lrs, [0,120000])

In [85]:
# Ablation 02, SEBNet BaselinePHead SGD, Pretrained, Batch Size 6, OHEM Cross Entropy - 78.98 Best mIoU
log_file_path = '/home/robert.breslin/alessandro/paper_2/mmsegmentation/work_dirs/sebnet_baseline-p-head_1xb6_cityscapes/20250825_134022/20250825_134022.log'
epochs, mIoU, mAcc, lrs = parse_log_file(log_file_path)
plot_accuracies_and_lr_interactive(epochs, mAcc, mIoU, lrs, [0,160000])

In [86]:
# Ablation 03, SEBNet BaselineDHead SGD, Pretrained, Batch Size 6, OHEM Cross Entropy, Edge Width 4, Edge Loss Weight 20.0 - 73.90 Best mIoU
log_file_path = '/home/robert.breslin/alessandro/paper_2/mmsegmentation/work_dirs/sebnet_baseline-d-head_1xb6_cityscapes/20250825_134116/20250825_134116.log'
epochs, mIoU, mAcc, lrs = parse_log_file(log_file_path)
plot_accuracies_and_lr_interactive(epochs, mAcc, mIoU, lrs, [0,160000])

In [87]:
# Ablation 19, SEBNet BaselinePDHead SGD, Pretrained, Batch Size 6, OHEM Cross Entropy - 79.57 Best mIoU
log_file_path = '/home/robert.breslin/alessandro/paper_2/mmsegmentation/work_dirs/sebnet_baseline-p-d-head_1xb6_cityscapes/20250825_134133/20250825_134133.log'
epochs, mIoU, mAcc, lrs = parse_log_file(log_file_path)
plot_accuracies_and_lr_interactive(epochs, mAcc, mIoU, lrs, [0,160000])

In [88]:
# Ablation 25, SEBNet BaselinePDBASHead SGD, Pretrained, Batch Size 6, OHEM Cross Entropy - 78.87 Best mIoU
log_file_path = '/home/robert.breslin/alessandro/paper_2/mmsegmentation/work_dirs/sebnet_baseline-p-d-bas-head_1xb6_cityscapes/20250825_134156/20250825_134156.log'
epochs, mIoU, mAcc, lrs = parse_log_file(log_file_path)
plot_accuracies_and_lr_interactive(epochs, mAcc, mIoU, lrs, [0,160000])

In [89]:
# Ablation 03, SEBNet BaselineDHead SGD, Pretrained, Batch Size 6, OHEM Cross Entropy, Edge Width 2, Edge Loss Weight 5.0 - 73.17 Best mIoU
log_file_path = '/home/robert.breslin/alessandro/paper_2/mmsegmentation/work_dirs/sebnet_baseline-d-head_1xb6_cityscapes/20250826_135921/20250826_135921.log'
epochs, mIoU, mAcc, lrs = parse_log_file(log_file_path)
plot_accuracies_and_lr_interactive(epochs, mAcc, mIoU, lrs, [0,160000])

In [90]:
# Ablation 09 - SEBNet BaselineCASENetHeadEarlierLayers SGD, Pretrained, Batch Size 6, OHEM Cross Entropy, Edge Width 2, Edge Loss Weight 5.0 - 73.45 Best mIoU
log_file_path = '/home/robert.breslin/alessandro/paper_2/mmsegmentation/work_dirs/sebnet_baseline-casenet-head-earlier-layers_1xb6_cityscapes/20250827_123728/20250827_123728.log'
epochs, mIoU, mAcc, lrs = parse_log_file(log_file_path)
plot_accuracies_and_lr_interactive(epochs, mAcc, mIoU, lrs, [0,160000])

In [91]:
# Ablation 12 - SEBNet BaselineDMultiLabelHeadEarlierLayers SGD, Pretrained, Batch Size 6, OHEM Cross Entropy, Edge Width 2, Edge Loss Weight 5.0 - 74.40 Best mIoU
log_file_path = '/home/robert.breslin/alessandro/paper_2/mmsegmentation/work_dirs/sebnet_baseline-d-multilabel-head-earlier-layers_1xb6_cityscapes/20250827_123750/20250827_123750.log'
epochs, mIoU, mAcc, lrs = parse_log_file(log_file_path)
plot_accuracies_and_lr_interactive(epochs, mAcc, mIoU, lrs, [0,160000])

In [92]:
# Ablation 10 - SEBNet BaselineDFFHeadEarlierLayers SGD, Pretrained, Batch Size 6, OHEM Cross Entropy, Edge Width 2, Edge Loss Weight 5.0 - 73.69 Best mIoU
log_file_path = '/home/robert.breslin/alessandro/paper_2/mmsegmentation/work_dirs/sebnet_baseline-dff-head-earlier-layers_1xb6_cityscapes/20250827_123829/20250827_123829.log'
epochs, mIoU, mAcc, lrs = parse_log_file(log_file_path)
plot_accuracies_and_lr_interactive(epochs, mAcc, mIoU, lrs, [0,160000])

In [93]:
# Ablation 08 - SEBNet BaselineDHeadEarlierLayers SGD, Pretrained, Batch Size 6, OHEM Cross Entropy, Edge Width 2, Edge Loss Weight 5.0 - 73.51 Best mIoU
log_file_path = '/home/robert.breslin/alessandro/paper_2/mmsegmentation/work_dirs/sebnet_baseline-d-head-earlier-layers_1xb6_cityscapes/20250827_130234/20250827_130234.log'
epochs, mIoU, mAcc, lrs = parse_log_file(log_file_path)
plot_accuracies_and_lr_interactive(epochs, mAcc, mIoU, lrs, [0,160000])

In [94]:
# SEBNet BaselineDSBDHead SGD, Pretrained, Batch Size 6, OHEM Cross Entropy, Edge Width 2, Edge Loss Weight 5.0 - XX.XX Best mIoU
log_file_path = '/home/robert.breslin/alessandro/paper_2/mmsegmentation/work_dirs/sebnet_baseline-d-sbd-head_1xb6_cityscapes/20250903_135626/20250903_135626.log'
epochs, mIoU, mAcc, lrs = parse_log_file(log_file_path)
plot_accuracies_and_lr_interactive(epochs, mAcc, mIoU, lrs, [0,160000])

In [95]:
# SEBNet BaselinePDSBDHead SGD, Pretrained, Batch Size 6, OHEM Cross Entropy, Edge Width 2, Edge Loss Weight 5.0 - XX.XX Best mIoU
log_file_path = '/home/robert.breslin/alessandro/paper_2/mmsegmentation/work_dirs/sebnet_baseline-p-d-sbd-head_1xb6_cityscapes/20250903_135655/20250903_135655.log'
epochs, mIoU, mAcc, lrs = parse_log_file(log_file_path)
plot_accuracies_and_lr_interactive(epochs, mAcc, mIoU, lrs, [0,160000])

In [96]:
# SEBNet BaselinePDSBDBASHead SGD, Pretrained, Batch Size 6, OHEM Cross Entropy, Edge Width 2, Edge Loss Weight 5.0 - XX.XX Best mIoU
log_file_path = '/home/robert.breslin/alessandro/paper_2/mmsegmentation/work_dirs/sebnet_baseline-p-d-sbd-bas-head_1xb6_cityscapes/20250903_135710/20250903_135710.log'
epochs, mIoU, mAcc, lrs = parse_log_file(log_file_path)
plot_accuracies_and_lr_interactive(epochs, mAcc, mIoU, lrs, [0,160000])

In [97]:
# SEBNet BaselinePDSBDBASWithSBDHead SGD, Pretrained, Batch Size 6, OHEM Cross Entropy, Edge Width 2, Edge Loss Weight 5.0 - XX.XX Best mIoU
log_file_path = '/home/robert.breslin/alessandro/paper_2/mmsegmentation/work_dirs/sebnet_baseline-p-d-sbd-basWithSBD-head_1xb6_cityscapes/20250903_135729/20250903_135729.log'
epochs, mIoU, mAcc, lrs = parse_log_file(log_file_path)
plot_accuracies_and_lr_interactive(epochs, mAcc, mIoU, lrs, [0,160000])

In [98]:
# Ablation 25 - SEBNet PIDNet SGD, Pretrained, Batch Size 6, OHEM Cross Entropy - 80.00 Best mIoU
log_file_path = '/home/robert.breslin/alessandro/paper_2/mmsegmentation/work_dirs/sebnet_baseline-p-d-bas-head_1xb6_cityscapes/20250823_235000/20250823_235000.log'
epochs, mIoU, mAcc, lrs = parse_log_file(log_file_path)
plot_accuracies_and_lr_interactive(epochs, mAcc, mIoU, lrs, [0,160000])

In [99]:
# Ablation 20 - SEBNet BaselineDSBDHead SGD, Pretrained, Batch Size 6, OHEM Cross Entropy, Edge Width 2, Edge Loss Weight 5.0 - 74.65 Best mIoU
log_file_path = '/home/robert.breslin/alessandro/paper_2/mmsegmentation/work_dirs/sebnet_baseline-d-sbd-head_1xb6_cityscapes/20250903_225043/20250903_225043.log'
epochs, mIoU, mAcc, lrs = parse_log_file(log_file_path)
plot_accuracies_and_lr_interactive(epochs, mAcc, mIoU, lrs, [0,160000])

In [100]:
# Ablation 26 - SEBNet BaselinePDSBDHead SGD, Pretrained, Batch Size 6, OHEM Cross Entropy, Edge Width 2, Edge Loss Weight 5.0 - 79.63 Best mIoU
log_file_path = '/home/robert.breslin/alessandro/paper_2/mmsegmentation/work_dirs/sebnet_baseline-p-d-sbd-head_1xb6_cityscapes/20250903_225120/20250903_225120.log'
epochs, mIoU, mAcc, lrs = parse_log_file(log_file_path)
plot_accuracies_and_lr_interactive(epochs, mAcc, mIoU, lrs, [0,160000])

In [101]:
# Ablation 27 - SEBNet BaselinePDSBDBASHead SGD, Pretrained, Batch Size 6, OHEM Cross Entropy, Edge Width 2, Edge Loss Weight 5.0 - 80.44 Best mIoU
log_file_path = '/home/robert.breslin/alessandro/paper_2/mmsegmentation/work_dirs/sebnet_baseline-p-d-sbd-bas-head_1xb6_cityscapes/20250903_225204/20250903_225204.log'
epochs, mIoU, mAcc, lrs = parse_log_file(log_file_path)
plot_accuracies_and_lr_interactive(epochs, mAcc, mIoU, lrs, [0,160000])

In [102]:
# SEBNet BaselinePDSBDBASWithSBDHead SGD, Pretrained, Batch Size 6, OHEM Cross Entropy, Edge Width 2, Edge Loss Weight 5.0 - 80.09 Best mIoU
log_file_path = '/home/robert.breslin/alessandro/paper_2/mmsegmentation/work_dirs/sebnet_baseline-p-d-sbd-basWithSBD-head_1xb6_cityscapes/20250903_225426/20250903_225426.log'
epochs, mIoU, mAcc, lrs = parse_log_file(log_file_path)
plot_accuracies_and_lr_interactive(epochs, mAcc, mIoU, lrs, [0,160000])

In [103]:
# PIDNet SGD, Pretrained, Batch Size 6, OHEM Cross Entropy - 80.00 Best mIoU
log_file_path = '/home/robert.breslin/alessandro/paper_2/mmsegmentation/work_dirs/pidnet-l_2xb6-120k_1024x1024-cityscapes/20250906_103110/20250906_103110.log'
epochs, mIoU, mAcc, lrs = parse_log_file(log_file_path)
plot_accuracies_and_lr_interactive(epochs, mAcc, mIoU, lrs, [0,240064])

In [104]:
# Ablation 25 - SEBNet BaselinePDBASHead SGD, Pretrained, Batch Size 6, OHEM Cross Entropy - 79.99 Best mIoU
log_file_path = '/home/robert.breslin/alessandro/paper_2/mmsegmentation/work_dirs/sebnet_baseline-p-d-bas-head_1xb6_cityscapes/20250911_103625/20250911_103625.log'
epochs, mIoU, mAcc, lrs = parse_log_file(log_file_path)
plot_accuracies_and_lr_interactive(epochs, mAcc, mIoU, lrs, [0,160000])

In [105]:
# Ablation 25 - SEBNet BaselinePDBASHead SGD, Pretrained, Batch Size 6, OHEM Cross Entropy - 80.47 Best mIoU
log_file_path = '/home/robert.breslin/alessandro/paper_2/mmsegmentation/work_dirs/sebnet_baseline-p-d-bas-head_1xb6_cityscapes/20250906_105242/20250906_105242.log'
epochs, mIoU, mAcc, lrs = parse_log_file(log_file_path)
plot_accuracies_and_lr_interactive(epochs, mAcc, mIoU, lrs, [0,240064])

In [106]:
# Ablation 24 - SEBNet BagBaselinePSBDBASHead SGD, Pretrained, Batch Size 6, OHEM Cross Entropy - 80.23 Best mIoU
log_file_path = '/home/robert.breslin/alessandro/paper_2/mmsegmentation/work_dirs/sebnet_baseline-p-sbd-bas-head-fused_1xb6_cityscapes/20250906_103054/20250906_103054.log'
epochs, mIoU, mAcc, lrs = parse_log_file(log_file_path)
plot_accuracies_and_lr_interactive(epochs, mAcc, mIoU, lrs, [0,240064])

In [107]:
# Ablation 28 - SEBNet BaselinePDSBDBASHead SGD, Mapillary, 480K Iters - 80.23 Best mIoU
log_file_path = '/home/robert.breslin/alessandro/paper_2/mmsegmentation/work_dirs/sebnet_baseline-p-d-sbd-bas-head_1xb6_mapillaryv2/20250909_160646/20250909_160646.log'
epochs, mIoU, mAcc, lrs = parse_log_file(log_file_path)
plot_accuracies_and_lr_interactive(epochs, mAcc, mIoU, lrs, [0,480000])

In [108]:
# Ablation 28 - SEBNet ConditionedBaselinePSBDBASHead SGD, Mapillary, 480K Iters - 80.23 Best mIoU
log_file_path = '/home/robert.breslin/alessandro/paper_2/mmsegmentation/work_dirs/sebnet_baseline-p-sbd-bas-head-conditioned_1xb6_mapillaryv2/20250909_160714/20250909_160714.log'
epochs, mIoU, mAcc, lrs = parse_log_file(log_file_path)
plot_accuracies_and_lr_interactive(epochs, mAcc, mIoU, lrs, [0,480000])

In [109]:
import re

# Function to parse the log file
def custom_parse_log_file(log_file_path):
    iterations = []
    mious = []
    maccs = []
    learning_rates = []

    last_iter = None
    last_lr = None

    with open(log_file_path, 'r') as f:
        for line in f:
            # Parse training iteration and learning rate (updated regex)
            train_match = re.search(r"Iter\(train\)\s*\[\s*(\d+)/\d+\]\s*base_lr:\s*[0-9\.e+-]+\s*lr:\s*([0-9\.e+-]+)", line)
            if train_match:
                last_iter = int(train_match.group(1))
                last_lr = float(train_match.group(2))
                continue

            # Parse validation metrics (mIoU and mAcc) - unchanged
            val_match = re.search(r"mIoU:\s*(\d+\.\d+)\s*mAcc:\s*(\d+\.\d+)", line)
            if val_match:
                if last_iter is not None:
                    iterations.append(last_iter)
                    mious.append(float(val_match.group(1)))
                    maccs.append(float(val_match.group(2)))
                    learning_rates.append(last_lr)

    return iterations, mious, maccs, learning_rates
# Ablation 28 - SEBNet BaselinePDSBDBASHead AdamW, Mapillary, 300K Iters, 4xb4 - XX.XX Best mIoU
log_file_path = "/home/robert.breslin/alessandro/paper_2/mmsegmentation/work_dirs/sebnet_baseline-p-d-sbd-bas-head_2xb8_mapillaryv2/20250917_125856/20250917_125856.log"
epochs, mIoU, mAcc, lrs = custom_parse_log_file(log_file_path)
plot_accuracies_and_lr_interactive(epochs, mAcc, mIoU, lrs, [0,300000])

In [110]:
# Ablation 01 - SEBNet BaselineHead SGD, Pretrained, Batch Size 6, OHEM Cross Entropy - 72.72 Best mIoU
log_file_path = '/home/robert.breslin/alessandro/paper_2/mmsegmentation/work_dirs/sebnet_baseline-head_1xb6_cityscapes_ohem/20250822_162921/20250822_162921.log'
epochs, mIoU, mAcc, lrs = parse_log_file(log_file_path)
plot_accuracies_and_lr_interactive(epochs, mAcc, mIoU, lrs, [0,160000])

In [111]:
# Ablation 02, SEBNet BaselinePHead SGD, Pretrained, Batch Size 6, OHEM Cross Entropy - 78.98 Best mIoU
log_file_path = '/home/robert.breslin/alessandro/paper_2/mmsegmentation/work_dirs/sebnet_baseline-p-head_1xb6_cityscapes/20250825_134022/20250825_134022.log'
epochs, mIoU, mAcc, lrs = parse_log_file(log_file_path)
plot_accuracies_and_lr_interactive(epochs, mAcc, mIoU, lrs, [0,160000])

In [112]:
# Ablation 03 - 160K Iters, Batch Size 6, Weight 0.4 - 74.51 Best mIoU
log_file_path = "/home/robert.breslin/alessandro/paper_2/mmsegmentation/work_dirs/sebnet_ablation03_1xb6-160k_cityscapes/20250928_154446/20250928_154446.log"
epochs, mIoU, mAcc, lrs = parse_log_file(log_file_path)
plot_accuracies_and_lr_interactive(epochs, mAcc, mIoU, lrs, [0,160000])

In [113]:
# Ablation 03 - 160K Iters, Batch Size 6, Weight 1.0 - 74.51 Best mIoU
log_file_path = "/home/robert.breslin/alessandro/paper_2/mmsegmentation/work_dirs/sebnet_ablation03_1xb6-160k_cityscapes/20250928_154446/20250928_154446.log"
epochs, mIoU, mAcc, lrs = parse_log_file(log_file_path)
plot_accuracies_and_lr_interactive(epochs, mAcc, mIoU, lrs, [0,160000])

In [114]:
# Ablation 04 - 160K Iters, Batch Size 6, Weight 0.4 - 73.43 Best mIoU
log_file_path = "/home/robert.breslin/alessandro/paper_2/mmsegmentation/work_dirs/sebnet_ablation04_1xb6-160k_cityscapes/20250928_154519/20250928_154519.log"
epochs, mIoU, mAcc, lrs = parse_log_file(log_file_path)
plot_accuracies_and_lr_interactive(epochs, mAcc, mIoU, lrs, [0,160000])

In [115]:
# Ablation 04 - 160K Iters, Batch Size 6, Weight 1.0 - 73.27 Best mIoU
log_file_path = "/home/robert.breslin/alessandro/paper_2/mmsegmentation/work_dirs/sebnet_ablation04_1xb6-160k_cityscapes/20250929_102345/20250929_102345.log"
epochs, mIoU, mAcc, lrs = parse_log_file(log_file_path)
plot_accuracies_and_lr_interactive(epochs, mAcc, mIoU, lrs, [0,160000])

In [116]:
# Ablation 05 - 160K Iters, Batch Size 6, Weight 0.4 - 73.91 Best mIoU
log_file_path = "/home/robert.breslin/alessandro/paper_2/mmsegmentation/work_dirs/sebnet_ablation05_1xb6-160k_cityscapes/20250928_154719/20250928_154719.log"
epochs, mIoU, mAcc, lrs = parse_log_file(log_file_path)
plot_accuracies_and_lr_interactive(epochs, mAcc, mIoU, lrs, [0,160000])

In [117]:
# Ablation 05 - 160K Iters, Batch Size 6, Weight 1.0 - 73.28 Best mIoU
log_file_path = "/home/robert.breslin/alessandro/paper_2/mmsegmentation/work_dirs/sebnet_ablation05_1xb6-160k_cityscapes/20250929_102433/20250929_102433.log"
epochs, mIoU, mAcc, lrs = parse_log_file(log_file_path)
plot_accuracies_and_lr_interactive(epochs, mAcc, mIoU, lrs, [0,160000])

In [118]:
# Ablation 41 - 160K Iters, Batch Size 6 - 74.69 Best mIoU
log_file_path = "/home/robert.breslin/alessandro/paper_2/mmsegmentation/work_dirs/sebnet_ablation41_1xb6-160k_cityscapes/20250928_154817/20250928_154817.log"
epochs, mIoU, mAcc, lrs = parse_log_file(log_file_path)
plot_accuracies_and_lr_interactive(epochs, mAcc, mIoU, lrs, [0,160000])

In [119]:
# Ablation 22 - SEBNet BaselineDSBDHead SGD, Pretrained, Batch Size 6, OHEM Cross Entropy, Edge Width 2, Edge Loss Weight 5.0 - 74.65 Best mIoU
log_file_path = '/home/robert.breslin/alessandro/paper_2/mmsegmentation/work_dirs/sebnet_baseline-d-sbd-head_1xb6_cityscapes/20250903_225043/20250903_225043.log'
epochs, mIoU, mAcc, lrs = parse_log_file(log_file_path)
plot_accuracies_and_lr_interactive(epochs, mAcc, mIoU, lrs, [0,160000])

In [120]:
# Ablation 51 - 160K Iters, Batch Size 6, [3, 3, 21, 3], [3, 3, 21], No PreHead - XX.XX Best mIoU
log_file_path = "/home/robert.breslin/alessandro/paper_2/mmsegmentation/work_dirs/sebnet_ablation51_1xb6-160k_cityscapes/20250929_101826/20250929_101826.log"
epochs, mIoU, mAcc, lrs = parse_log_file(log_file_path)
plot_accuracies_and_lr_interactive(epochs, mAcc, mIoU, lrs, [0,160000])

In [121]:
# Ablation 51 - 160K Iters, Batch Size 6, [3, 3, 21, 3], [3, 3, 21], Yes PreHead - XX.XX Best mIoU
log_file_path = "/home/robert.breslin/alessandro/paper_2/mmsegmentation/work_dirs/sebnet_ablation51_1xb6-160k_cityscapes/20250929_112032/20250929_112032.log"
epochs, mIoU, mAcc, lrs = parse_log_file(log_file_path)
plot_accuracies_and_lr_interactive(epochs, mAcc, mIoU, lrs, [0,160000])

In [122]:
# Ablation 51 - 160K Iters, Batch Size 6, [3, 3, 21, 3], [3, 21, 3], No PreHead - XX.XX Best mIoU
log_file_path = "/home/robert.breslin/alessandro/paper_2/mmsegmentation/work_dirs/sebnet_ablation51_1xb6-160k_cityscapes/20250929_124210/20250929_124210.log"
epochs, mIoU, mAcc, lrs = parse_log_file(log_file_path)
plot_accuracies_and_lr_interactive(epochs, mAcc, mIoU, lrs, [0,160000])

In [123]:
# Ablation 51 - 160K Iters, Batch Size 6, [3, 3, 21, 3], [3, 21, 3], Yes PreHead - XX.XX Best mIoU
log_file_path = "/home/robert.breslin/alessandro/paper_2/mmsegmentation/work_dirs/sebnet_ablation51_1xb6-160k_cityscapes/20250929_134243/20250929_134243.log"
epochs, mIoU, mAcc, lrs = parse_log_file(log_file_path)
plot_accuracies_and_lr_interactive(epochs, mAcc, mIoU, lrs, [0,160000])

In [124]:
# Ablation 51 - 160K Iters, Batch Size 6, [3, 3, 9, 3], [3, 3, 9], LW 1.0, No PreHead - XX.XX Best mIoU
log_file_path = "/home/robert.breslin/alessandro/paper_2/mmsegmentation/work_dirs/sebnet_ablation51_1xb6-160k_cityscapes/20250929_153022/20250929_153022.log"
epochs, mIoU, mAcc, lrs = parse_log_file(log_file_path)
plot_accuracies_and_lr_interactive(epochs, mAcc, mIoU, lrs, [0,160000])

In [125]:
# Ablation 51 - 160K Iters, Batch Size 6, [3, 3, 9, 3], [3, 3, 9], LW 5.0, No PreHead - XX.XX Best mIoU
log_file_path = "/home/robert.breslin/alessandro/paper_2/mmsegmentation/work_dirs/sebnet_ablation51_1xb6-160k_cityscapes/20250929_160648/20250929_160648.log"
epochs, mIoU, mAcc, lrs = parse_log_file(log_file_path)
plot_accuracies_and_lr_interactive(epochs, mAcc, mIoU, lrs, [0,160000])

In [126]:
# Ablation 51 - 160K Iters, Batch Size 6, [3, 3, 9, 3], [3, 3, 9], LW 10.0, No PreHead - XX.XX Best mIoU
log_file_path = "/home/robert.breslin/alessandro/paper_2/mmsegmentation/work_dirs/sebnet_ablation51_1xb6-160k_cityscapes/20250929_164650/20250929_164650.log"
epochs, mIoU, mAcc, lrs = parse_log_file(log_file_path)
plot_accuracies_and_lr_interactive(epochs, mAcc, mIoU, lrs, [0,160000])

In [127]:
# Ablation 51 - 160K Iters, Batch Size 6, [3, 3, 9, 3], [3, 3, 9], LW 0.4, No PreHead - XX.XX Best mIoU
log_file_path = "/home/robert.breslin/alessandro/paper_2/mmsegmentation/work_dirs/sebnet_ablation51_1xb6-160k_cityscapes/20250929_172325/20250929_172325.log"
epochs, mIoU, mAcc, lrs = parse_log_file(log_file_path)
plot_accuracies_and_lr_interactive(epochs, mAcc, mIoU, lrs, [0,160000])

In [128]:
# Ablation 51 - 160K Iters, Batch Size 6, [3, 3, 9, 3], [3, 3, 9], LW 1.0, xlarge arch, No PreHead - XX.XX Best mIoU
log_file_path = "/home/robert.breslin/alessandro/paper_2/mmsegmentation/work_dirs/sebnet_ablation51_1xb6-160k_cityscapes/20250929_181638/20250929_181638.log"
epochs, mIoU, mAcc, lrs = parse_log_file(log_file_path)
plot_accuracies_and_lr_interactive(epochs, mAcc, mIoU, lrs, [0,160000])

In [129]:
# Ablation 51 - 160K Iters, Batch Size 6, [3, 4, 4, 2], [3, 4, 4], LW 1.0, Wider channels, No PreHead - 73.66 Best mIoU
log_file_path = "/home/robert.breslin/alessandro/paper_2/mmsegmentation/work_dirs/sebnet_ablation51_1xb6-160k_cityscapes/20250929_191108/20250929_191108.log"
epochs, mIoU, mAcc, lrs = parse_log_file(log_file_path)
plot_accuracies_and_lr_interactive(epochs, mAcc, mIoU, lrs, [0,160000])

In [130]:
# Ablation 02, SEBNet BaselinePHead SGD, Pretrained, Batch Size 6, OHEM Cross Entropy - 78.98 Best mIoU
log_file_path = '/home/robert.breslin/alessandro/paper_2/mmsegmentation/work_dirs/sebnet_baseline-p-head_1xb6_cityscapes/20250825_134022/20250825_134022.log'
epochs, mIoU, mAcc, lrs = parse_log_file(log_file_path)
plot_accuracies_and_lr_interactive(epochs, mAcc, mIoU, lrs, [0,160000])

In [131]:
# Ablation 51 - 160K Iters, Batch Size 6, [3, 3, 9, 3], [3, 3, 9], Yes PreHead - XX.XX Best mIoU
log_file_path = "/home/robert.breslin/alessandro/paper_2/mmsegmentation/work_dirs/sebnet_ablation51_1xb6-160k_cityscapes/20250929_112032/20250929_112032.log"
epochs, mIoU, mAcc, lrs = parse_log_file(log_file_path)
plot_accuracies_and_lr_interactive(epochs, mAcc, mIoU, lrs, [0,160000])

In [132]:
# Ablation 51 - 160K Iters, Batch Size 6, [3, 3, 9, 3], [3, 9, 3 ], No PreHead - XX.XX Best mIoU
log_file_path = "/home/robert.breslin/alessandro/paper_2/mmsegmentation/work_dirs/sebnet_ablation51_1xb6-160k_cityscapes/20250929_112032/20250929_112032.log"
epochs, mIoU, mAcc, lrs = parse_log_file(log_file_path)
plot_accuracies_and_lr_interactive(epochs, mAcc, mIoU, lrs, [0,160000])

In [133]:
# Ablation 51 - 160K Iters, Batch Size 6, [3, 3, 9, 3], [3, 9, 3], Yes PreHead - XX.XX Best mIoU
log_file_path = "/home/robert.breslin/alessandro/paper_2/mmsegmentation/work_dirs/sebnet_ablation51_1xb6-160k_cityscapes/20250929_112032/20250929_112032.log"
epochs, mIoU, mAcc, lrs = parse_log_file(log_file_path)
plot_accuracies_and_lr_interactive(epochs, mAcc, mIoU, lrs, [0,160000])

In [134]:
# Ablation 22 - SEBNet BaselineDSBDHead SGD, Pretrained, Batch Size 6, OHEM Cross Entropy, Edge Width 2, Edge Loss Weight 5.0 - 74.65 Best mIoU
log_file_path = '/home/robert.breslin/alessandro/paper_2/mmsegmentation/work_dirs/sebnet_baseline-d-sbd-head_1xb6_cityscapes/20250903_225043/20250903_225043.log'
epochs, mIoU, mAcc, lrs = parse_log_file(log_file_path)
plot_accuracies_and_lr_interactive(epochs, mAcc, mIoU, lrs, [0,160000])

In [135]:
# Ablation 41 - 160K Iters, Batch Size 6, HED: 1.0, SBD: 1.0 - 74.29 Best mIoU
log_file_path = "/home/robert.breslin/alessandro/paper_2/mmsegmentation/work_dirs/sebnet_ablation41_1xb6-160k_cityscapes/20251001_151814/20251001_151814.log"
epochs, mIoU, mAcc, lrs = parse_log_file(log_file_path)
plot_accuracies_and_lr_interactive(epochs, mAcc, mIoU, lrs, [0,160000])

In [136]:
# Ablation 41 - 160K Iters, Batch Size 6, HED: 1.0, SBD: 5.0 - 74.22 Best mIoU
log_file_path = "/home/robert.breslin/alessandro/paper_2/mmsegmentation/work_dirs/sebnet_ablation41_1xb6-160k_cityscapes/20251001_151845/20251001_151845.log"
epochs, mIoU, mAcc, lrs = parse_log_file(log_file_path)
plot_accuracies_and_lr_interactive(epochs, mAcc, mIoU, lrs, [0,160000])

In [137]:
# Ablation 41 - 160K Iters, Batch Size 6, HED: 1.0, SBD: 10.0 - 74.59 Best mIoU
log_file_path = "/home/robert.breslin/alessandro/paper_2/mmsegmentation/work_dirs/sebnet_ablation41_1xb6-160k_cityscapes/20251001_151903/20251001_151903.log"
epochs, mIoU, mAcc, lrs = parse_log_file(log_file_path)
plot_accuracies_and_lr_interactive(epochs, mAcc, mIoU, lrs, [0,160000])

In [138]:
# Ablation 41 - 160K Iters, Batch Size 6, HED: 1.0, SBD: 20.0 - 74.15 Best mIoU
log_file_path = "/home/robert.breslin/alessandro/paper_2/mmsegmentation/work_dirs/sebnet_ablation41_1xb6-160k_cityscapes/20251001_151949/20251001_151949.log"
epochs, mIoU, mAcc, lrs = parse_log_file(log_file_path)
plot_accuracies_and_lr_interactive(epochs, mAcc, mIoU, lrs, [0,160000])

In [139]:
# Ablation 41 - 160K Iters, Batch Size 6, HED: 5.0, SBD: 1.0 - 73.67 Best mIoU
log_file_path = "/home/robert.breslin/alessandro/paper_2/mmsegmentation/work_dirs/sebnet_ablation41_1xb6-160k_cityscapes/20251005_100330/20251005_100330.log"
epochs, mIoU, mAcc, lrs = parse_log_file(log_file_path)
plot_accuracies_and_lr_interactive(epochs, mAcc, mIoU, lrs, [0,160000])

In [140]:
# Ablation 41 - 160K Iters, Batch Size 6, HED: 5.0, SBD: 5.0 - 74.69 Best mIoU
log_file_path = "/home/robert.breslin/alessandro/paper_2/mmsegmentation/work_dirs/sebnet_ablation41_1xb6-160k_cityscapes/20251005_100355/20251005_100355.log"
epochs, mIoU, mAcc, lrs = parse_log_file(log_file_path)
plot_accuracies_and_lr_interactive(epochs, mAcc, mIoU, lrs, [0,160000])

In [141]:
# Ablation 41 - 160K Iters, Batch Size 6, HED: 5.0, SBD: 10.0 - 74.44 Best mIoU
log_file_path = "/home/robert.breslin/alessandro/paper_2/mmsegmentation/work_dirs/sebnet_ablation41_1xb6-160k_cityscapes/20251005_100418/20251005_100418.log"
epochs, mIoU, mAcc, lrs = parse_log_file(log_file_path)
plot_accuracies_and_lr_interactive(epochs, mAcc, mIoU, lrs, [0,160000])

In [142]:
# Ablation 41 - 160K Iters, Batch Size 6, HED: 5.0, SBD: 20.0 - 73.81 Best mIoU
log_file_path = "/home/robert.breslin/alessandro/paper_2/mmsegmentation/work_dirs/sebnet_ablation41_1xb6-160k_cityscapes/20251005_100503/20251005_100503.log"
epochs, mIoU, mAcc, lrs = parse_log_file(log_file_path)
plot_accuracies_and_lr_interactive(epochs, mAcc, mIoU, lrs, [0,160000])

In [143]:
# Ablation 41 - 160K Iters, Batch Size 6, HED: 10.0, SBD: 1.0 - 74.04 Best mIoU
log_file_path = "/home/robert.breslin/alessandro/paper_2/mmsegmentation/work_dirs/sebnet_ablation41_1xb6-160k_cityscapes/20251007_163903/20251007_163903.log"
epochs, mIoU, mAcc, lrs = parse_log_file(log_file_path)
plot_accuracies_and_lr_interactive(epochs, mAcc, mIoU, lrs, [0,160000])

In [144]:
# Ablation 41 - 160K Iters, Batch Size 6, HED: 10.0, SBD: 5.0 - 74.44 Best mIoU
log_file_path = "/home/robert.breslin/alessandro/paper_2/mmsegmentation/work_dirs/sebnet_ablation41_1xb6-160k_cityscapes/20251007_163922/20251007_163922.log"
epochs, mIoU, mAcc, lrs = parse_log_file(log_file_path)
plot_accuracies_and_lr_interactive(epochs, mAcc, mIoU, lrs, [0,160000])

In [145]:
# Ablation 41 - 160K Iters, Batch Size 6, HED: 10.0, SBD: 10.0 - 74.61 Best mIoU
log_file_path = "/home/robert.breslin/alessandro/paper_2/mmsegmentation/work_dirs/sebnet_ablation41_1xb6-160k_cityscapes/20251007_163943/20251007_163943.log"
epochs, mIoU, mAcc, lrs = parse_log_file(log_file_path)
plot_accuracies_and_lr_interactive(epochs, mAcc, mIoU, lrs, [0,160000])

In [146]:
# Ablation 41 - 160K Iters, Batch Size 6, HED: 10.0, SBD: 20.0 - 73.45 Best mIoU
log_file_path = "/home/robert.breslin/alessandro/paper_2/mmsegmentation/work_dirs/sebnet_ablation41_1xb6-160k_cityscapes/20251007_164003/20251007_164003.log"
epochs, mIoU, mAcc, lrs = parse_log_file(log_file_path)
plot_accuracies_and_lr_interactive(epochs, mAcc, mIoU, lrs, [0,160000])

In [147]:
# Ablation 43 - 160K Iters, Batch Size 6 - 79.50 Best mIoU
log_file_path = "/home/robert.breslin/alessandro/paper_2/mmsegmentation/work_dirs/sebnet_ablation43_1xb6-160k_cityscapes/20251009_190552/20251009_190552.log"
epochs, mIoU, mAcc, lrs = parse_log_file(log_file_path)
plot_accuracies_and_lr_interactive(epochs, mAcc, mIoU, lrs, [0,160000])

In [148]:
# Ablation 44 - 160K Iters, Batch Size 6 - 79.82 Best mIoU
log_file_path = "/home/robert.breslin/alessandro/paper_2/mmsegmentation/work_dirs/sebnet_ablation44_1xb6-160k_cityscapes/20251009_190607/20251009_190607.log"
epochs, mIoU, mAcc, lrs = parse_log_file(log_file_path)
plot_accuracies_and_lr_interactive(epochs, mAcc, mIoU, lrs, [0,160000])

In [149]:
# Ablation 45 - 160K Iters, Batch Size 6 - 79.72 Best mIoU
log_file_path = "/home/robert.breslin/alessandro/paper_2/mmsegmentation/work_dirs/sebnet_ablation45_1xb6-160k_cityscapes/20251009_190632/20251009_190632.log"
epochs, mIoU, mAcc, lrs = parse_log_file(log_file_path)
plot_accuracies_and_lr_interactive(epochs, mAcc, mIoU, lrs, [0,160000])

In [150]:
# Ablation 47 - 240K Iters, Batch Size 6 - 80.05 Best mIoU
log_file_path = "/home/robert.breslin/alessandro/paper_2/mmsegmentation/work_dirs/sebnet_ablation47_1xb6-160k_cityscapes/20251009_190652/20251009_190652.log"
epochs, mIoU, mAcc, lrs = parse_log_file(log_file_path)
plot_accuracies_and_lr_interactive(epochs, mAcc, mIoU, lrs, [0,160000])

In [151]:
# Ablation 39 - 240K Iters, Batch Size 6 - 80.05 Best mIoU
log_file_path = "/home/robert.breslin/alessandro/paper_2/mmsegmentation/work_dirs/sebnet_ablation39_1xb6-160k_cityscapes/20251012_183707/20251012_183707.log"
epochs, mIoU, mAcc, lrs = parse_log_file(log_file_path)
plot_accuracies_and_lr_interactive(epochs, mAcc, mIoU, lrs, [0,241000])

In [152]:
# Ablation 47 - 240K Iters, Batch Size 6 - 80.05 Best mIoU
log_file_path = "/home/robert.breslin/alessandro/paper_2/mmsegmentation/work_dirs/sebnet_ablation47_1xb6-160k_cityscapes/20251012_183601/20251012_183601.log"
epochs, mIoU, mAcc, lrs = parse_log_file(log_file_path)
plot_accuracies_and_lr_interactive(epochs, mAcc, mIoU, lrs, [0,241000])

In [153]:
# Ablation 48 - 240K Iters, Batch Size 6 - 80.05 Best mIoU
log_file_path = "/home/robert.breslin/alessandro/paper_2/mmsegmentation/work_dirs/sebnet_ablation48_1xb6-160k_cityscapes/20251012_183311/20251012_183311.log"
epochs, mIoU, mAcc, lrs = parse_log_file(log_file_path)
plot_accuracies_and_lr_interactive(epochs, mAcc, mIoU, lrs, [0,241000])

In [154]:
# Ablation 49 - 240K Iters, Batch Size 6 - 80.05 Best mIoU
log_file_path = "/home/robert.breslin/alessandro/paper_2/mmsegmentation/work_dirs/sebnet_ablation49_1xb6-160k_cityscapes/20251012_183214/20251012_183214.log"
epochs, mIoU, mAcc, lrs = parse_log_file(log_file_path)
plot_accuracies_and_lr_interactive(epochs, mAcc, mIoU, lrs, [0,241000])